In [1]:
#######################

In [16]:
import pandas as pd
import numpy as np
import re 

In [17]:
times=pd.read_csv("timesData.csv")
cwur=pd.read_csv("cwurData.csv")
shanghai=pd.read_csv("shanghaiData.csv")

In [70]:
#Si convertono i - in valori mancanti
times["income"]=times["income"].replace('-', np.NaN)

#Si trasforma income da stringa a float
times["income"]=times["income"].astype(float)



For each university, extract from the times dataset the most recent and the least recent data, obtaining two separate dataframes

In [71]:
#Si raggruppa per Università e si prende l'anno massimo 
timesRecent=times.iloc[times.groupby('university_name')['year'].idxmax()][['university_name','year','income','world_rank',
                                              'num_students','female_male_ratio','country']]


In [72]:
timesRecent.head() #Si stampano i risultati 

,university_name,year,income,world_rank,num_students,female_male_ratio,country
2405,AGH University of Science and Technology,2016,NaN,601-800,"35,569",-,Poland
2003,Aalborg University,2016,43.7,201-250,"17,422",48 : 52,Denmark
2056,Aalto University,2016,61.6,251-300,"16,099",32 : 68,Finland
1908,Aarhus University,2016,68.3,=106,"23,895",54 : 46,Denmark
2105,Aberystwyth University,2016,31.3,301-350,"9,252",48 : 52,United Kingdom


In [73]:
#Si raggruppa per Università e si prende l'anno minimo 
timesLast=times.iloc[times.groupby('university_name')['year'].idxmin()][['university_name','year','income','world_rank',
                                              'num_students','female_male_ratio','country']]


In [74]:
timesLast.head() #Si stampano i risultati 

,university_name,year,income,world_rank,num_students,female_male_ratio,country
2405,AGH University of Science and Technology,2016,NaN,601-800,"35,569",-,Poland
501,Aalborg University,2012,36.4,301-350,"17,422",48 : 52,Denmark
502,Aalto University,2012,61.9,301-350,"16,099",32 : 68,Finland
166,Aarhus University,2011,61.5,167,"23,895",54 : 46,Denmark
476,Aberystwyth University,2012,35.5,276-300,"9,252",48 : 52,United Kingdom


For each university, compute the improvement in income between the least recent and the most recent data points

In [75]:
#Si uniscono le tabelle precedentemente ottenute
improvement=pd.merge(timesRecent, timesLast, on=["university_name"], suffixes = ['_last', '_recent'])

#Si crea una nuova colonna "improvement" con il miglioramento del guadagno
improvement["improvement"]=(improvement["income_last"]-improvement["income_recent"])
improvement.head()

,university_name,year_last,income_last,world_rank_last,num_students_last,female_male_ratio_last,country_last,year_recent,income_recent,world_rank_recent,num_students_recent,female_male_ratio_recent,country_recent,improvement
0,AGH University of Science and Technology,2016,NaN,601-800,"35,569",-,Poland,2016,NaN,601-800,"35,569",-,Poland,NaN
1,Aalborg University,2016,43.7,201-250,"17,422",48 : 52,Denmark,2012,36.4,301-350,"17,422",48 : 52,Denmark,7.3
2,Aalto University,2016,61.6,251-300,"16,099",32 : 68,Finland,2012,61.9,301-350,"16,099",32 : 68,Finland,-0.3
3,Aarhus University,2016,68.3,=106,"23,895",54 : 46,Denmark,2011,61.5,167,"23,895",54 : 46,Denmark,6.8
4,Aberystwyth University,2016,31.3,301-350,"9,252",48 : 52,United Kingdom,2012,35.5,276-300,"9,252",48 : 52,United Kingdom,-4.2


Find the university with the largest increase computed in the previous point

In [76]:
improvement[improvement['improvement']>=improvement['improvement'].max()]

,university_name,year_last,income_last,world_rank_last,num_students_last,female_male_ratio_last,country_last,year_recent,income_recent,world_rank_recent,num_students_recent,female_male_ratio_recent,country_recent,improvement
428,TU Dresden,2016,99.7,=158,"35,487",42 : 58,Germany,2012,31.9,251-275,"35,487",42 : 58,Germany,67.8


4. For each ranking, consider only the most recent data point. For each university, compute the maximum difference between the rankings (e.g. for Aarhus University the value is 122-73=49). Notice that some rankings are expressed as a range

In [77]:
#Prendiamo i dati più recenti per tutti i rank


###############
shanghaiRank = shanghai.iloc[shanghai.groupby('university_name')['year'].idxmax()][['university_name','year','world_rank']]
shanghaiRank.rename(columns={'year':'year_shanghai','world_rank':'world_rank_shanghai'}, inplace=True)

###############
timesRank = timesRecent[['university_name','year','world_rank']]
timesRank.rename(columns={'year':'year_time','world_rank':'world_rank_time'}, inplace=True)

###############
cwurRank = cwur.iloc[cwur.groupby('institution')['year'].idxmax()][['institution','year','world_rank']]
cwurRank.rename(columns={'institution':'university_name','year':'year_cwur','world_rank':'world_rank_cwur'}, inplace=True)


/Users/mario/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [78]:
#Uniamo tutte le tabelle. how='outer' così da considerare tutti i casi

cwurTimes = pd.merge(cwurRank, timesRank, on='university_name', how='outer')
rank = pd.merge(cwurTimes, shanghaiRank, on='university_name', how='outer')


In [79]:
#Implementiamo una funzione per estrarre le differenze massime

def maxDiffRank(rank):
    #Suffisso per scorrere sulle colonne di rank
    namesRank = ['time','shanghai','cwur'] 
    diffRank=[]
    for name in namesRank:
        if pd.notna(rank['world_rank_'+name]): #Non si considerano i valori nulli
            if re.search('-',str(rank['world_rank_'+name])) is not None: #Se c'è un range si calcola la media
                estremi=rank['world_rank_'+name].split('-') #Separiamo gli estremi
                diffRank.append((int(estremi[0])+int(estremi[1]))/2) #Calcolo media
            else:
                #Prendiamo il valore numerico del rank
                diffRank.append(int(re.search('(\d+)',str(rank['world_rank_'+name])).group(0)))
    #Restituiamo la differenza massima            
    return max(diffRank)-min(diffRank)   


In [80]:
#Applichiamo la funzione appena creata
rank['maxDiffRank'] = rank.apply(maxDiffRank, axis=1)
rank.head()


,university_name,year_cwur,world_rank_cwur,year_time,world_rank_time,year_shanghai,world_rank_shanghai,maxDiffRank
0,AGH University of Science and Technology,2015.0,782.0,2016.0,601-800,NaN,NaN,81.5
1,Aalborg University,2015.0,565.0,2016.0,201-250,2015.0,301-400,339.5
2,Aalto University,2015.0,421.0,2016.0,251-300,2015.0,401-500,175.0
3,Aarhus University,2015.0,122.0,2016.0,=106,2015.0,73,49.0
4,Aberystwyth University,2015.0,814.0,2016.0,301-350,NaN,NaN,488.5


In [81]:
#Controllo
rank[rank['university_name']=='Aarhus University']

,university_name,year_cwur,world_rank_cwur,year_time,world_rank_time,year_shanghai,world_rank_shanghai,maxDiffRank
3,Aarhus University,2015.0,122.0,2016.0,=106,2015.0,73,49.0


Consider only the most recent data point of the times dataset. Compute the number of male and female students for each country.

In [95]:
#Convertiamo i simboli "-" in valori mancanti
timesRecent["female_male_ratio"]=times["female_male_ratio"].replace('-', np.NaN)

#Si eliminano i valori nulli, num valori nulli, perchè
timesRecentNonull = timesRecent.dropna(subset=['female_male_ratio','num_students'])

#Convertiamo stringa in intero
timesRecentNonull['num_students'] = timesRecentNonull['num_students'].str.replace(',', '').astype(int)


/Users/mario/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [96]:
#Si creano due gruppi float con percentuali di femmine e maschi
genere = timesRecentNonull['female_male_ratio'].str.extract(r'(?P<female_percentage>^\d+) : (?P<male_percentage>\d+$)').astype(float)

#Si crea una colonna con il numero di femmine
timesRecentNonull['female']=((genere.female_percentage/100)*timesRecentNonull['num_students']).round(0)

#Si crea una colonna con il numero di maschi
timesRecentNonull['male']=((genere.male_percentage/100)*timesRecentNonull['num_students']).round(0)


/Users/mario/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/mario/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [97]:
timesRecentNonull[['university_name','male','female']]

,university_name,male,female
2003,Aalborg University,9059.0,8363.0
2056,Aalto University,10947.0,5152.0
1908,Aarhus University,10992.0,12903.0
2105,Aberystwyth University,4811.0,4441.0
2404,Adam Mickiewicz University,11784.0,28849.0
...,...,...,...
1856,École Normale Supérieure,1296.0,1104.0
2013,École Normale Supérieure de Lyon,1131.0,1087.0
1904,École Polytechnique,1992.0,437.0
1833,École Polytechnique Fédérale de Lausanne,7056.0,2610.0


In [94]:
#times_most_recent_nonull['female_percentage']=genere.female_percentage
#times_most_recent_nonull['male_percentage']=genere.male_percentage
timesRecentNonull.groupby('country')['female'].sum()
timesRecentNonull.groupby('country')['male'].sum()

country
Argentina                     41182.0
Australia                    321640.0
Austria                       66113.0
Bangladesh                    41393.0
Belarus                        9084.0
                              ...    
Uganda                        18670.0
Ukraine                       19250.0
United Arab Emirates           4931.0
United Kingdom               613028.0
United States of America    1556324.0
Name: male, Length: 70, dtype: float64